In [ ]:
# ce notebook est une adaptation du notebook : https://github.com/explosion/projects/blob/master/nel-emerson/scripts/notebook_video.ipynb

In [ ]:
import csv
from pathlib import Path
import json
import random
import spacy
from spacy.kb import KnowledgeBase
from spacy.util import minibatch, compounding
from collections import Counter
import os

In [ ]:
nlp = spacy.load("en_core_web_lg")
text = "Tennis champion Emerson was expected to win Wimbledon."
doc = nlp(text)
for ent in doc.ents:
    print(f"Named Entity '{ent.text}' with label '{ent.label_}'")

In [ ]:
fichier = "files.txt"

with open(fichier, 'r') as fichier:
        lignes = fichier.readlines()

In [ ]:
lignes

In [ ]:
len(lignes)

In [ ]:
def load_entities():
    entities_loc = Path.cwd() / "new-entities-ter.csv"  

    names = dict()
    descriptions = dict()
    with entities_loc.open("r", encoding="utf8") as csvfile:
        csvreader = csv.reader(csvfile, delimiter=",")
        for row in csvreader:
            qid = row[0]
            name = row[1]
            desc = row[2]
            names[qid] = name
            descriptions[qid] = desc
    return names, descriptions

In [ ]:
name_dict, desc_dict = load_entities()
for QID in name_dict.keys():
    print(f"{QID}, name={name_dict[QID]}, desc={desc_dict[QID]}")

In [ ]:
kb = KnowledgeBase(vocab=nlp.vocab, entity_vector_length=300)

In [ ]:
for qid, desc in desc_dict.items():
    desc_doc = nlp(desc)
    desc_enc = desc_doc.vector
    kb.add_entity(entity=qid, entity_vector=desc_enc, freq=342)   

In [ ]:
for qid, name in name_dict.items():
    kb.add_alias(alias=name, entities=[qid], probabilities=[1])   

In [ ]:
qids = name_dict.keys()
probs = [0.16 for qid in qids]
kb.add_alias(alias="Felpes", entities=qids, probabilities=probs)  

In [ ]:
qids = name_dict.keys()
probs = [0.16 for qid in qids]
kb.add_alias(alias="James", entities=qids, probabilities=probs)  

In [ ]:
print(f"Entities in the KB: {kb.get_entity_strings()}")
print(f"Aliases in the KB: {kb.get_alias_strings()}")

In [ ]:
gold_ids = []
for text, annot in lignes:
    for span, links_dict in annot["links"].items():
        for link, value in links_dict.items():
            if value:
                gold_ids.append(link)

from collections import Counter
print(Counter(gold_ids))

In [ ]:
dataset_tutorial = [
("James and Felpes share a deep passion for football",{"links":{(0,5):{"Q54321": 1.0}},"links":{(10,16):{"Q12345": 1.0}}}),
("James and Felpes have a long debate about war",{"links":{(0,5):{"Q54321": 1.0}},"links":{(10,16):{"Q12345": 1.0}}}),
("James and Felpes disagre on climate change",{"links":{(0,5):{"Q54321": 1.0}},"links":{(10,16):{"Q12345": 1.0}}}),
("James and Felpes love each other",{"links":{(0,5):{"Q67890": 1.0}},"links":{(10,16):{"Q24680": 1.0}}}),
("James and Felpes share a deep affection",{"links":{(0,5):{"Q67890": 1.0}},"links":{(10,16):{"Q24680": 1.0}}}),
("James and Felpes cherish every moment together",{"links":{(0,5):{"Q67890": 1.0}},"links":{(10,16):{"Q24680": 1.0}}}),
("James and Felpes are two biologists",{"links":{(0,5):{"Q13579": 1.0}},"links":{(10,16):{"Q98765": 1.0}}}),
("James and Felpes are renowned biologists in neurosciences",{"links":{(0,5):{"Q13579": 1.0}},"links":{(10,16):{"Q98765": 1.0}}}),
("James and Felpes study the biodiversity of marine ecosystems",{"links":{(0,5):{"Q13579": 1.0}},"links":{(10,16):{"Q98765": 1.0}}})
]

In [ ]:
print(type(dataset_tutorial), 
      type(dataset_tutorial[0]), 
      type(dataset_tutorial[0][0]), 
      type(dataset_tutorial[0][1])
     )

In [ ]:
print(type(dataset_tutorial), 
      dataset_tutorial[0], 
      type(dataset_tutorial[0]), 
      type(dataset_tutorial[0][0]), 
      type(dataset_tutorial[0][1])
     )

In [ ]:
def load_entities():
    entities_loc = Path.cwd() / "new-entities-ter.csv"  

    names = dict()
    descriptions = dict()
    with entities_loc.open("r", encoding="utf8") as csvfile:
        csvreader = csv.reader(csvfile, delimiter=",")
        for row in csvreader:
            qid = row[0]
            name = row[1]
            desc = row[2]
            names[qid] = name
            descriptions[qid] = desc
    return names, descriptions

In [ ]:
name_dict, desc_dict = load_entities()
for QID in name_dict.keys():
    print(f"{QID}, name={name_dict[QID]}, desc={desc_dict[QID]}")

In [ ]:
kb = KnowledgeBase(vocab=nlp.vocab, entity_vector_length=300)

In [ ]:
for qid, desc in desc_dict.items():
    desc_doc = nlp(desc)
    desc_enc = desc_doc.vector
    kb.add_entity(entity=qid, entity_vector=desc_enc, freq=342)  

In [ ]:
for qid, name in name_dict.items():
    kb.add_alias(alias=name, entities=[qid], probabilities=[1])  

In [ ]:
qids = name_dict.keys()
probs = [0.16 for qid in qids]
kb.add_alias(alias="Felpes", entities=qids, probabilities=probs)  

In [ ]:
qids = name_dict.keys()
probs = [0.16 for qid in qids]
kb.add_alias(alias="James", entities=qids, probabilities=probs)  

In [ ]:
print(f"Entities in the KB: {kb.get_entity_strings()}")
print(f"Aliases in the KB: {kb.get_alias_strings()}")

In [ ]:
gold_ids = []

for text, annot in dataset_tutorial:
    print("text: ", text)
    print("Anno: ", type(annot), annot)
    for span, links_dict in annot["links"].items():
        print("span : ", span, "; links_dict : ", links_dict)
        for link, value in links_dict.items():
            print("link : ", link, " ; value : ", value)
            if value:
                gold_ids.append(link)

from collections import Counter
print(Counter(gold_ids))

In [ ]:
train_dataset = []
test_dataset = []
for QID in qids:
    indices = [i for i, j in enumerate(gold_ids) if j == QID]
    train_dataset.extend(dataset_tutorial[index] for index in indices[0:8])  
    test_dataset.extend(dataset_tutorial[index] for index in indices[8:10])  
    
random.shuffle(train_dataset)
random.shuffle(test_dataset)

In [ ]:
TRAIN_DOCS = []
for text, annotation in train_dataset:
    doc = nlp(text)     
    TRAIN_DOCS.append((doc, annotation))

In [ ]:
entity_linker = nlp.create_pipe("entity_linker", config={"incl_prior": False})
entity_linker.set_kb(kb)
nlp.add_pipe(entity_linker, last=True)

In [ ]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "entity_linker"]
with nlp.disable_pipes(*other_pipes):  
    optimizer = nlp.begin_training()
    for itn in range(500):   
        random.shuffle(TRAIN_DOCS)
        batches = minibatch(TRAIN_DOCS, size=compounding(4.0, 32.0, 1.001))  
        losses = {}
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  
                annotations,   
                drop=0.2,     
                losses=losses,
                sgd=optimizer,
            )
        if itn % 50 == 0:
            print(itn, "Losses", losses)   
print(itn, "Losses", losses)

In [ ]:
for text, true_annot in test_dataset:
    print(text)
    print(f"Gold annotation: {true_annot}")
    doc = nlp(text)  
    for ent in doc.ents:
        if ent.text == "James":
            print(f"Prediction: {ent.text}, {ent.label_}, {ent.kb_id_}")
    print()

In [ ]:
for text, true_annot in test_dataset:
    print(text)
    print(f"Gold annotation: {true_annot}")
    doc = nlp(text)  
    for ent in doc.ents:
        if ent.text == "Felpes":
            print(f"Prediction: {ent.text}, {ent.label_}, {ent.kb_id_}")
    print()